# Numerical Methods for Manu Body Physics, Assignment #4

Yoav Zack, ID 211677398

In [ ]:
using LinearAlgebra
using Statistics
using Plots
using LsqFit
using Printf
using BenchmarkTools
using SignalAnalysis
theme(:dracula)

## Question 1: 2D XY model and the Kosterlitz-Thouless transition

We want to implement a Monte-Carlo algorithm using Wolff Clusters for the 2D XY Model. In order to do so, we need the following functions:
1. `WolffUpdate()` - Updates a grid of spins 
2. `selectCluster()` - Selects a cluster according to Wolff method
3. `nearestNeighboors()` - Returns a list of nearest neighboors of a given site assuming given boundary conditions.

It will be easier for us to work if some of the data will be stores ina a `Grid` structure, as follows:

In [ ]:
struct Grid
    N::Integer
    grid::Matrix{Real}
    J::Real
    β::Real
    bc::String
    
    function Grid(N::Integer, J::Real, T::Real, bc::String)
        @assert N > 0 "Grid size must be positive integer"
        @assert J > 0 "Interaction coefficient J must be positive"
        @assert T > 0 "Temperature must be positive"

        if !(bc in ["periodic", "closed"])
            error("Boundacy conditions are either periodic or closed")
        end
        
        β = 1. / T
        grid = (2*π) .* rand(N,N)
        new(N, grid, J, β, bc)
    end
end

We will also define a simple function which plots the grid data ina a heatmap:

In [ ]:
function showGrid(grid::Matrix, β::Real)
    N = size(grid)[1]
    h = heatmap(grid, c=:cyclic_mygbm_30_95_c78_n256_s25, colorbar=nothing)
    heatmap!(h, xlim=(1,N), ylim=(1,N), xticks=0, yticks=0)
    heatmap!(h, aspect_ratio=:equal, frame=:on)
    heatmap!(h, title=@sprintf("N=%d, β=%.2f", N, β))
    return h
end

Now, we can start working on the functions mentioned above. First, the `nearestNeighboors()` function:

In [ ]:
function nearestNeighboors(i::Integer, j::Integer, grid::Grid)
    N = grid.N
    if grid.bc == "periodic"
        i = (i-1 % N) + 1
        j = (j-1 % N) + 1
    end

    @assert i >= 1 && j >= 1 && i <= N && j <= N "Invalid spin index"
    neighboors = [(i+1,j), (i,j+1), (i-1,j), (i,j-1)]
    
    if grid.bc == "periodic"
        return [mod.(a .- 1, N) .+ 1 for a in neighboors]
    elseif grid.bc == "closed"
        return [(x,y) for (x,y) in neighboors if (x>=1) && (y>=1) && (x<=N) && (y<=N)]
    else
        error("Invalid boundary condition in nearestNeighboors()")
    end
end

Notice that we solved it for both periodic and closed boundary conditions. Next, we work on the Wolff part - selecting a cluster. For it (and the next section), we will use the following identities:
$$
\begin{array}{rl}
\vec{S}_{i} & =\left(\cos\theta_{i},\sin\theta_{i}\right)\\
\hat{e} & =\left(\cos\theta_{e},\sin\theta_{e}\right)\\
\vec{S}_{i}\cdot\hat{e} & =\cos\theta_{i}\cos\theta_{e}+\sin\theta_{i}\sin\theta_{e}=\cos\left(\theta_{i}-\theta_{e}\right)\\
\vec{S}_{i}-2\left(\vec{S}_{i}\cdot\hat{e}\right)\hat{e} & =\left(\cos\theta_{i},\sin\theta_{i}\right)-2\cos\left(\theta_{i}-\theta_{e}\right)\left(\cos\theta_{e},\sin\theta_{j}\right)\\
 & =\left(-\cos\left(\theta_{i}-2\theta_{e}\right),\sin\left(\theta_{i}-2\theta_{e}\right)\right)
\end{array}
$$
Thus the function for selecting a cluster is:

In [ ]:
function selectCluster(θe::Real, i::Integer, j::Integer, grid::Grid)
    cluster = [(i,j)]
    queue = nearestNeighboors(i, j, grid)

    θi = grid.grid[i,j]
    prodi = cos(θi - θe)
    while !isempty(queue)
        s = pop!(queue)
        if s in cluster
            continue
        end
        θj = grid.grid[s[1], s[2]]
        prodj = cos(θj - θe)
        p = 1 - exp(min(0, -2*grid.β*grid.J*prodi*prodj))
        if p > rand()
            append!(cluster, [s])
            append!(queue, nearestNeighboors(s[1], s[2], grid))
        end
    end

    return cluster
end

This gives us a list of indices to by sliced by. Next, we write a function which does a single Wolff spin flip according to the function given in the assignment and the previously done calculations:

In [ ]:
function iterationWolff!(grid::Grid)
    i = rand(1:grid.N)
    j = rand(1:grid.N)

    θe = 2*π*rand()
    cluster = selectCluster(θe, i, j, grid)
    
    for s in cluster
        grid.grid[s[1], s[2]] = mod(grid.grid[s[1], s[2]] - 2*θe, 2*π)
    end
end

Let's benchmark our function to see if it is any good:

In [ ]:
N = 32
J = 1.0
T = 0.1
grid = Grid(N, J, T, "periodic")
@benchmark iterationWolff!(grid)

And the final result does make sense, at least on the visual aspect (the benchmark performs many iterations on the same grid, so we get a relatively valid sample after it is finished):

In [ ]:
showGrid(grid.grid, 1/T)

Now we want to verify whether the autocorrelation and equlibration time for various system sizes and temperatures. We define a function to perform a complete simulation:

In [ ]:
function WolffXY(N::Integer, J::Real, T::Real, Nsw::Integer, bc::String, saveConfigs=false)
    grid = Grid(N, J, T, bc)

    m = zeros(Nsw)
    m[1] = magnetizationXY(grid)
    
    if saveConfigs
        c = zeros(N,N,Nsw)
        c[:,:,1] = deepcopy(grid.grid)
    end

    for i in 2:Nsw
        iterationWolff!(grid)
        m[i] = magnetizationXY(grid)
        if saveConfigs
            c[:,:,i] = deepcopy(grid.grid)
        end
    end

    if saveConfigs
        return m, c
    else
        return m
    end
end

Where `magnetizationXY()` is a function which calculates the magnitude of total magnetization in the grid:

In [ ]:
function magnetizationXY(grid::Grid)
    vecs = [cos.(grid.grid), sin.(grid.grid)]
    m = norm(mean.(vecs))
    return m
end

In order to estimate the autocorrelation time we will define the error function from the tutorial (TODO)

In [ ]:
function calculateError(m, Neq, Nsw)
    err = Float64[]
    Nsw2 = 2^Int(floor(log(2, Nsw-Neq))) # closest power of 2 smaller than number of sweeps after equilibration
    ml = m[end-Nsw2+1:end]
    min_size = 2^5
    while length(ml) > min_size
        push!(err, std(ml)/sqrt(length(ml)-1))
        ml = map(j -> mean(ml[2*j-1:2*j]), range(1,stop=div(length(ml),2)) )
    end
    return err
end

In [ ]:
# physical constants
Nlist = 8:8:32
Tlist = [0.1, 1.0, 1.5]
J = 1

# iteration count
iterTotal = 100

# prepare for smoothing
winlength = 11
iterTotal += winlength
kernel = vcat(ones(winlength)./winlength, zeros(iterTotal-winlength))

# log arrays
Marr = zeros(length(Nlist), length(Tlist), iterTotal)
plist = []
hlist = []

# loop over all required temperatures and sizes
for (Tind, T) in enumerate(Tlist)
    for (Nind, N) in enumerate(Nlist)
        # simulate
        m, c = WolffXY(N, J, T, iterTotal, "periodic", true)
        
        # plot final state and magnetization log, as well as a smoothed version of it
        h = showGrid(c[:,:,end], 1/T)
        p = plot(m[1:end-winlength], title=@sprintf("N=%d, T=%.1f", N, T), label=nothing)
        plot!(p, circconv(kernel, m)[1:end-winlength], label=nothing, c=:red)

        # log plots to plot together later
        push!(plist, p)
        push!(hlist, h)
    end
end

In [ ]:
plot_size = (200*length(Nlist), 200*length(Tlist))
l = (length(Tlist), length(Nlist))
magplot = plot(plist..., size=plot_size, layout=l, plot_title="Magnetization Magnitude", ylims=(0, 1))
mapplot = plot(hlist..., size=plot_size, layout=l, plot_title="Final State")
display(magplot)
display(mapplot)